In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-15"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
3603,2024-04-15,Romênia Liga Nacional Feminina,13:00,CSM Constanta F,Arad F,1.35,3.02,138.5,1.85,1.85,0.0,0.00,0.00,OI5mqZMh,0.740741,0.331126,0.540541,0.540541,0.071867,474.630,875.133668,1.843823,2.4,1.341641,0.559017,81.65,5.736,10.211093,1.780177,73.0,300.902,268.602899,0.892659,2.4,1.341641,0.559017,504.72,4.058,3.188082,0.785629,39.0,71,72,1.15,7.01,172.890,687.856,0.540443,0.000000,NaN,5.17,1.034,0.338491,0.000000,0.0,0.000000,38.36,7.672,0.263295,0.000000,0.0,0.000000
3604,2024-04-15,Lituânia Lkl,12:50,Nevezis,Siauliai,1.97,1.76,176.5,1.87,1.79,1.5,1.83,1.62,hjVyjZZJ,0.507614,0.568182,0.534759,0.558659,0.075796,295.840,172.740542,0.583899,0.6,1.341641,2.236068,532.03,3.636,2.001582,0.550490,-42.0,206.130,103.205774,0.500683,0.6,1.341641,2.236068,122.50,2.654,1.705295,0.642538,-27.0,83,98,6.41,1.25,262.066,219.776,0.079621,0.030912,0.086083,0.13,0.026,37.307692,0.408531,0.5,0.091469,-0.93,-0.186,-4.086022,0.000000,0.0,0.000000
3605,2024-04-15,Lituânia Nkl,13:00,BC Olimpas Palanga,Suduva-Mantinga,1.46,2.59,154.5,1.85,1.85,0.0,0.00,0.00,z3UZnc63,0.684932,0.386100,0.540541,0.540541,0.071032,121.766,21.573838,0.177175,1.2,1.643168,1.369306,103.74,1.422,0.141315,0.099378,18.0,89.966,22.128106,0.245961,1.2,1.643168,1.369306,72.54,1.250,0.190263,0.152210,30.0,78,62,1.33,1.17,110.696,97.942,0.394583,0.000000,NaN,-2.57,-0.514,-0.894942,0.821899,0.6,-0.221899,0.56,0.112,14.196429,0.698805,0.6,-0.098805
3606,2024-04-15,México Cibacopa,23:15,Frayles de Guasave,Angeles CDMX,2.68,1.43,180.5,1.80,1.88,0.0,0.00,0.00,xWSCdxa7,0.373134,0.699301,0.555556,0.531915,0.072435,199.566,66.701889,0.334235,0.6,1.341641,2.236068,136.80,2.418,1.116723,0.461837,-36.0,114.170,13.383598,0.117225,1.8,1.643168,0.912871,133.40,1.292,0.108259,0.083792,30.0,80,92,1.71,1.45,196.326,168.202,0.430114,0.030744,NaN,1.40,0.280,6.000000,0.553044,0.4,-0.153044,3.01,0.602,0.714286,0.000000,0.0,0.000000
3607,2024-04-15,Nova Zelândia Nbl,04:30,Southland Sharks,Wellington Saints,4.75,1.16,181.5,1.84,1.86,0.0,0.00,0.00,zuEHI9VN,0.210526,0.862069,0.543478,0.537634,0.072595,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,80,98,2.10,1.50,0.000,0.000,0.859057,0.007644,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
3608,2024-04-15,Romênia Divisão A,11:00,CSM Focsani,Valcea,2.39,1.53,172.5,1.81,1.89,0.0,0.00,0.00,6swxZ6Yh,0.418410,0.653595,0.552486,0.529101,0.072005,279.504,162.756320,0.582304,1.2,1.643168,1.369306,358.20,3.180,1.708450,0.537248,-46.0,125.504,35.228668,0.280698,1.8,1.643168,0.912871,135.42,1.316,0.346814,0.263537,16.0,90,111,3.98,1.22,237.854,224.322,0.310261,0.030578,NaN,1.41,0.282,4.929078,0.000000,0.0,0.000000,-0.78,-0.156,-3.397436,0.000000,0.0,0.000000
3609,2024-04-15,Romênia Divisão A,12:00,Laguna Bucuresti,SCM Craiova,5.29,1.13,144.5,1.84,1.86,0.0,0.00,0.00,MkwORkRP,0.189036,0.884956,0.543478,0.537634,0.073992,598.058,107.960950,0.180519,0.0,0.000000,NaN,527.65,9.044,2.338467,0.258566,-75.0,121.664,59.156845,0.486231,3.0,0.000000,0.000000,77.70,1.462,0.656635,0.449135,71.0,61,74,8.65,1.05,376.852,183.984,0.916375,0.007644,NaN,2.28,0.456,9.407895,0.000000,0.0,0.000000,0.81,0.162,0.802469,0.462386,0.7,0.2376

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
